# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
w_data = pd.read_csv('output_data/Weather_data.csv')
w_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pangnirtung,66.1451,-65.7125,-27.00,69.0,20.0,1.68,CA,1.615659e+09
1,hilo,19.7297,-155.0900,21.00,78.0,75.0,2.22,US,1.615659e+09
2,fortuna,40.5982,-124.1573,11.11,81.0,90.0,3.09,US,1.615659e+09
3,adrar,20.5022,-10.0711,27.35,9.0,81.0,3.75,MR,1.615659e+09
4,boguchany,58.3814,97.4531,-17.14,96.0,22.0,2.73,RU,1.615659e+09
...,...,...,...,...,...,...,...,...,...
570,sturgis,41.7992,-85.4192,9.44,23.0,1.0,3.60,US,1.615659e+09
571,parana,-31.7320,-60.5238,33.89,35.0,38.0,1.34,AR,1.615660e+09
572,najran,17.4924,44.1277,26.00,19.0,36.0,3.09,SA,1.615660e+09
573,goderich,43.7501,-81.7165,0.00,55.0,24.0,6.69,CA,1.615660e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
# Configure gmaps
gmaps.configure(api_key=g_key)


# define weights
locations = w_data[["Lat", "Lng"]].astype(float)

humid = w_data['Humidity'].astype(float)


# Set up heatmap
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3
                                )

# Center the chart
fig = gmaps.figure(center = [0,0], zoom_level =2)

# Add heat layer
fig.add_layer(heat_layer)


# Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Max temp lower than 80 but higher than 70

# Wind speed less than 10mph

# zero cloudiness


0      156.2
1      172.4
2      177.8
3       48.2
4      204.8
       ...  
570     73.4
571     95.0
572     66.2
573    131.0
574    113.0
Name: Humidity, Length: 575, dtype: float64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
